In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import pprint

# 파이썬에서 인터넷을 연결하기 위해 urllib 패키지 사용. 
# urlopen함수는 지정한 url과 소켓 통신을 할 수 있도록 자동 연결해 줌

In [72]:
def get_product(KEY, STAT_CD, PERIOD, START_DATE, END_DATE):
    # 
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/{}/xml/kr/1/0000/{}/{}/{}/{}/' \
    .format(KEY          # 인증키
            , STAT_CD    # 추출할 통계지표의 코드
            , PERIOD    # 기간 단위
            , START_DATE # 데이터 시작일
            , END_DATE)  # 데이터 종료일

    response = requests.get(url).content.decode('utf-8')
    
    xml_obj = BeautifulSoup(response, 'lxml-xml')

    rows = xml_obj.findAll("row")
    return rows

In [81]:
# 파라미터 정의
# 추출하고자 하는 통계지표 disc type - {통계지표 코드: 통계지표명}
data_dict = {'521Y001' : '뉴스'}

# 인증키
KEY = '' # 인증키

# 그외 파라미터
PERIOD = 'D'
START_DATE = '19900101'
END_DATE = '20230315'

In [76]:
# API의 반환(출력)값 중 저장하고자 하는 항목(item) 리스트
item_list = ['STAT_CODE'    # 통계표코드
             , 'STAT_NAME'  # 통계명
             , 'ITEM_CODE1' # 통계항목1코드
             , 'ITEM_NAME1' # 통계항목명1
             , 'ITEM_CODE2' # 통계항목2코드
             , 'ITEM_NAME2' # 통계항목명2
             , 'ITEM_CODE3' # 통계항목3코드
             , 'ITEM_NAME3' # 통계항목명3
             , 'UNIT_NAME'  # 단위
             , 'TIME'       # 시점
             , 'DATA_VALUE' # 값
            ]

# 결과치를 담을 빈 리스트 생성
result_list = list()

# API를 순차적으로 호출하고 값을 담는 for loop 구문
for k in data_dict.keys():
    rows = get_product(KEY, k, PERIOD, START_DATE, END_DATE)
    print(len(rows)) # 수집해야 할 데이터의 row가 총 몇 개인지 출력
    
    for p in range(0, len(rows)):
        info_list = list()
        
        for i in item_list:
            try:
                individual_info = rows[p].find(i).text # 만약 반환 중 error가 발생하면
            except:
                individual_info = "" # 해당 항목은 공란으로 채운다
        
            info_list.append(individual_info)
        result_list.append(info_list)
    result_list

6648


In [77]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime

# 결과 리스트를 DataFrame으로 변환 + 컬럼명 지정
result_df = DataFrame(result_list, columns=['통계표코드'
                                            , '통계명'
                                            , '통계항목1코드'
                                            , '통계항목명1'
                                            , '통계항목2코드'
                                            , '통계항목명2'
                                            , '통계항목3코드'
                                            , '통계항목명3'
                                            , '단위'
                                            , '시점'
                                            , '값'
                                           ]).drop_duplicates() # 중복된 row 제거

result_df

,통계표코드,통계명,통계항목1코드,통계항목명1,통계항목2코드,통계항목명2,통계항목3코드,통계항목명3,단위,시점,값
0,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20050101,95.34
1,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20050102,95.8
2,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20050103,97.25
3,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20050104,101.1
4,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20050105,101.53
...,...,...,...,...,...,...,...,...,...,...,...
6643,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20230311,92.68
6644,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20230312,92.62
6645,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20230313,92.81
6646,521Y001,6.4. 뉴스심리지수(실험적 통계),A001,뉴스심리지수,,,,,,20230314,89.93


In [ ]:
# 엑셀로 저장
date = datetime.today().strftime('%Y-%m-%d')

result_df.drop_duplicates() \
      .to_excel(date+'_korea bank indicators.xlsx', index = False)